# GRAPE for systems with Dissipation

In [1]:
# ruff: noqa

In [2]:
import jax.numpy as jnp
from feedback_grape.grape import *
from feedback_grape.utils.gates import *
from feedback_grape.utils.operators import *
from feedback_grape.utils.states import *
from feedback_grape.utils.superoperator import *
from feedback_grape.utils.tensor import *

In [28]:
Sx = sigmax()
Sy = sigmay()
Sz = sigmaz()
Sm = sigmam()
Si = identity(2) @ identity(2).conj().T

had_gate = hadamard().conj().T
# Hamiltonian
Del = 0.1  # Tunnelling term
wq = 1.0  # Energy of the 2-level system.
H0 = 0.5 * wq * sigmaz() + 0.5 * Del * sigmax()

# Amplitude damping#
# Damping rate:
gamma = 0.1
L0 = liouvillian(H0, [jnp.sqrt(gamma) * Sm])

Hs = [Sx, Sy, Sz]  # Hamiltonian controls

# sigma X control
LC_x = liouvillian(Sx)
# sigma Y control
LC_y = liouvillian(Sy)
# sigma Z control
LC_z = liouvillian(Sz)

# Drift
drift = L0
# Controls - different combinations can be tried
ctrls = [LC_z, LC_x]
# Number of ctrls
n_ctrls = len(ctrls)

# start point for the map evolution
E0 = sprepost(Si, Si)

# target for map evolution
E_targ = sprepost(had_gate, had_gate)

In [29]:
# Number of time slots
n_ts = 10
# Time allowed for the evolution
evo_time = 2

In [21]:
result = optimize_pulse(
    drift,
    ctrls,
    E0,
    E_targ,
    n_ts,
    evo_time,
    type="superoperator",
    optimizer="adam",
    convergence_threshold=1e-16,
    max_iter=1000,
    learning_rate=0.01,
    propcomp="memory-efficient"
)

In [22]:
result

result(control_amplitudes=Array([[ 0.28188125, -0.17997523],
       [ 0.37115688,  0.03234856],
       [ 0.43806319, -0.08515268],
       [ 0.41188119, -0.03062981],
       [ 0.16348436, -0.03787871],
       [ 0.16660085, -0.13161439],
       [ 0.45696588,  0.07392594],
       [ 0.37858587, -0.14173639],
       [ 0.03038538, -0.05791306],
       [ 0.15622052,  0.05829588]], dtype=float64), final_fidelity=Array(0.99588702, dtype=float64), iterations=1000, final_operator=Array([[ 6.69782544e-01+3.86725987e-21j,  6.00043094e-04+2.15282298e-02j,
         6.00043094e-04-2.15282298e-02j,  6.92509743e-04+7.88577408e-21j],
       [ 2.45747277e-04-1.68366390e-02j, -8.18093101e-01+4.26748819e-05j,
        -6.27002426e-04-3.45964855e-05j, -7.31435297e-04+2.62957844e-02j],
       [ 2.45747277e-04+1.68366390e-02j, -6.27002426e-04+3.45964855e-05j,
        -8.18093101e-01-4.26748819e-05j, -7.31435297e-04-2.62957844e-02j],
       [ 3.30217456e-01-3.86725987e-21j, -6.00043094e-04-2.15282298e-02j,
     

In [ ]:
result_2 = optimize_pulse(
    H_drift=H0,
    H_control=Hs,
    U_0=Si,
    C_target=had_gate,
    num_t_slots=n_ts,
    total_evo_time=evo_time,
    c_ops=[jnp.sqrt(gamma) * Sm],
    type="density",
    optimizer="l-bfgs",
    convergence_threshold=1e-16,
    max_iter=1000,
    learning_rate=0.01,
    propcomp="memory-efficient"
)

In [31]:
result_2

result(control_amplitudes=Array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]], dtype=float64), final_fidelity=Array(0., dtype=float64), iterations=2, final_operator=Array([[nan+nanj, nan+nanj],
       [nan+nanj, nan+nanj]], dtype=complex128))

In [16]:
print(fidelity(C_target= E_targ, U_final=result.final_operator, type="superoperator"))

0.995887021027934
